In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from datetime import timedelta
from sklearn import preprocessing
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
# set(test_df.columns) - set(train_df.columns)
# remove MiscFeature and keep only MiscVal
# features to mean encode: MSZoning, Neighborhood
# GarageYrBlt gives you an interesting insight about the house age. There are 81 missing value.
# train_df.YrSold - train_df.YearBuilt can be an interesting feature
# train_df.YrSold - train_df.YearRemodAdd
# sum of bedrooms: BedroomAbvGr + Bedroom
# train_df.KitchenAbvGr * train_df.KitchenQual
# train_df.1stFlrSF + train_df.2ndFlrSF - train_df.LowQualFinSF : LowQualFinSF should be remove because it contains only 23 non zero row
# from attribute related to pool we can only keep PoolQC because all the others are full of nan values.
# 3SsnPorch: lot of 0.
# TotalBsmtSF - BsmtUnfSF , then remove BsmtUnfSF
# GarageCars and GarageCars are correlated. We can keep only one of them
# FullBath + 0.5 halfbath
target_col="HotelPrice_0"
cat_cols=set() # use s.add(4) and s.update([1,2]) to append multiple elt.
# handling missing data
def check_missing_data(df,col_prefix="train_"):
    nbr_missing_values_col=df.isnull().sum().sort_values(ascending=False)
    percentage_missing_values_col=(df.isnull().sum() / df.isnull().count()).sort_values(ascending=False)
    missing_data=pd.concat([nbr_missing_values_col,percentage_missing_values_col],axis=1,keys=[col_prefix+"total",col_prefix+"percent"])
    res=missing_data[missing_data[col_prefix+"total"]>0]
    return res

def check_missing_data_for_train_and_test(train,test):
    missing_train=check_missing_data(train,"train_")
    missing_test=check_missing_data(test,"test_")
    all_missing_data=pd.concat([missing_train,missing_test],axis=1).sort_values(by='train_total',ascending=False)
    return all_missing_data

def fill_missing_value_with_None(train,test,cols):
    train[cols].fillna('None',inplace=True)
    test[cols].fillna('None',inplace=True)
    return train,test
    
def fill_missing_value_with_most_freq_value(train,test,cols):
    for col in cols:
        most_freq_value=train[col].mode()[0]
        train[col].fillna(most_freq_value,inplace=True)
        test[col].fillna(most_freq_value,inplace=True)
    return train,test

def fill_missing_value_with_mean(train,test,cols):
    for col in cols:
        mean=train[col].mean()
        train[col].fillna(mean,inplace=True)
        test[col].fillna(mean,inplace=True)
    return train,test

def create_new_column_indicating_missing_value(train,test,threshold=0.02):
    mis=check_missing_data(train)
    cols_with_missing=mis[mis.train_percent>threshold].index
    print("nbr of columns with a lot of missing values: ",len(list(cols_with_missing)))
    for col in cols_with_missing:
        train[col + '_was_missing'] = train[col].isnull()
        test[col + '_was_missing'] = test[col].isnull()
    useful_cols=train.T.drop_duplicates().index
    train=train[useful_cols]
    test=test[train.drop(columns=target_col).columns]
    return train,test

# EDA
def get_columns_types(df):
    object_cols=df.select_dtypes(include=['object']).columns
    int_cols=df.select_dtypes(include=['int64']).columns
    float_cols=df.select_dtypes(include=['float64']).columns
    print("nbr of object cols:",len(object_cols))
    print("nbr of int cols:",len(int_cols))
    print("nbr of float cols",len(float_cols))
    return object_cols,int_cols,float_cols

def describe_dataset(data):
    ncats = []
    ncats10 = []
    ncats100 = []
    nsamples_median = []
    X_col_names = list(data.columns)
    
    #X_col_names.remove(target_col)
    print('Number of samples: ', data.shape[0])
    for col in X_col_names:
        counts = data.groupby([col])[col].count()
        ncats.append(len(counts))
        ncats10.append(len(counts[counts<10]))
        ncats100.append(len(counts[counts<100]))
        nsamples_median.append(counts.median())
    percent_cats=np.array(ncats) / data.shape[0]
    data_review_df = pd.DataFrame({'Column':X_col_names, 'Number of categories':ncats,
                                   'percent_cat':percent_cats, 
                                   'Categories with < 10 samples':ncats10,
                                   'Categories with < 100 samples':ncats100,
                                   'Median samples in category':nsamples_median})
    data_review_df = data_review_df.loc[:, ['Column', 'Number of categories','percent_cat',
                                             'Median samples in category',
                                             'Categories with < 10 samples',
                                             'Categories with < 100 samples']]
    return data_review_df.sort_values(by=['Number of categories'], ascending=True)

def describe_target_value(target):
    print(target.describe())
    from scipy.stats import norm
    sns.distplot(target,fit=norm)
    print("Skewness: %f" % target.skew())
    print("Kurtosis: %f" % target.kurt())

# Encoding
def encode_ordinal_feature(train,test,column_name,mapper):
    train[column_name]=train[column_name].replace(mapper)
    test[column_name]=test[column_name].replace(mapper)
    
def label_encode_categorical_features(train,test,cat_columns,target_col):
    ntrain = train.shape[0]
    ntest = test.shape[0]
    target=train[target_col]
    all_data = pd.concat((train, test)).reset_index(drop=True)
    all_data.drop([target_col], axis=1, inplace=True)
    for col in cat_columns:
        le = preprocessing.LabelEncoder()
        all_data[col]=le.fit_transform(all_data[col].fillna("None"))
    train=all_data[:ntrain]
    train[target_col]=target
    test=all_data[ntrain:]
    test.reset_index(inplace=True,drop=True)
    return train,test
def  time_features1(x,date_col,extension): 
    x[date_col]=pd.to_datetime(x[date_col],format='%d/%m/%Y')
    x["dayofweek_"+extension]=x[date_col].dt.dayofweek
    x["dayofyear_"+extension]=x[date_col].dt.dayofyear
    x["dayofmonth_"+extension]=x[date_col].dt.day
    x["year_woy_"+extension]=x[date_col].dt.year.astype(str)+x[date_col].dt.weekofyear.astype(str)
    x["is_weekend_"+extension]=x["dayofweek_"+extension].apply( lambda x : 1 if x  in [5,6] else 0 )
    x["year_"+extension]=x[date_col].dt.year
    x["quarter_"+extension]=x[date_col].dt.quarter
    x["month_"+extension]=x[date_col].dt.month
    
    x["dayofweek_"+extension]=pd.to_numeric(x[date_col])
    x["dayofyear_"+extension]=pd.to_numeric(x[date_col].dt.dayofyear)
    x["dayofmonth_"+extension]=pd.to_numeric(x[date_col].dt.day)
    x["year_woy_"+extension]=pd.to_numeric(x[date_col].dt.year.astype(str)+x[date_col].dt.weekofyear.astype(str))
    x["is_weekend_"+extension]=pd.to_numeric(x["dayofweek_"+extension].apply( lambda x : 1 if x  in [5,6] else 0 ))
    x["year_"+extension]=pd.to_numeric(x[date_col].dt.year)
    x["quarter_"+extension]=pd.to_numeric(x[date_col].dt.quarter)
    x["month_"+extension]=pd.to_numeric(x[date_col].dt.month)
    return x 
def  time_features2(x,date_col,extension): 
    x[date_col]=pd.to_datetime(x[date_col],format='%d-%m-%Y')
    x["dayofweek_"+extension]=x[date_col].dt.dayofweek
    x["dayofyear_"+extension]=x[date_col].dt.dayofyear
    x["dayofmonth_"+extension]=x[date_col].dt.day
    x["year_woy_"+extension]=x[date_col].dt.year.astype(str)+x[date_col].dt.weekofyear.astype(str)
    x["is_weekend_"+extension]=x["dayofweek_"+extension].apply( lambda x : 1 if x  in [5,6] else 0 )
    x["year_"+extension]=x[date_col].dt.year
    x["quarter_"+extension]=x[date_col].dt.quarter
    x["month_"+extension]=x[date_col].dt.month
    x["dayofweek_"+extension]=pd.to_numeric(x[date_col])
    x["dayofyear_"+extension]=pd.to_numeric(x[date_col].dt.dayofyear)
    x["dayofmonth_"+extension]=pd.to_numeric(x[date_col].dt.day)
    x["year_woy_"+extension]=pd.to_numeric(x[date_col].dt.year.astype(str)+x[date_col].dt.weekofyear.astype(str))
    x["is_weekend_"+extension]=pd.to_numeric(x["dayofweek_"+extension].apply( lambda x : 1 if x  in [5,6] else 0 ))
    x["year_"+extension]=pd.to_numeric(x[date_col].dt.year)
    x["quarter_"+extension]=pd.to_numeric(x[date_col].dt.quarter)
    x["month_"+extension]=pd.to_numeric(x[date_col].dt.month)
    return x
def decale_date(x,decal):
    today = datetime.strptime(x, '%d/%m/%Y')
    yesterday =today - timedelta(days=decal)
    return yesterday.strftime('%d/%m/%Y')

def generate_descriptive_stats(train_df,test_df,col,list_of_cols):
    train_df[col+"_mean"]=train_df[list_of_cols].mean(axis=1)
    test_df[col+"_mean"]=test_df[list_of_cols].mean(axis=1)
    train_df[col+"_max"]=train_df[list_of_cols].max(axis=1)
    test_df[col+"_max"]=test_df[list_of_cols].max(axis=1)
    train_df[col+"_min"]=train_df[list_of_cols].min(axis=1)
    test_df[col+"_min"]=test_df[list_of_cols].min(axis=1)
    train_df[col+"_median"]=train_df[list_of_cols].median(axis=1)
    test_df[col+"_median"]=test_df[list_of_cols].median(axis=1)
    train_df[col+"_std"]=train_df[list_of_cols].std(axis=1)
    test_df[col+"_std"]=test_df[list_of_cols].std(axis=1)
    
    return train_df,test_df

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df
    
    
columns_to_drop=["Date","Extraction_Date","Row_id","Hotel","Date_id"]

In [3]:
train_df=pd.read_csv("training_set.csv.zip")
test_df=pd.read_csv("testing_set.csv.zip")
sample_submission=pd.read_csv("submission_sample.csv.zip")
print("----------Shape of Data----------")
print(train_df.shape)
print("-----------Information-----------")
print(train_df.info())
print("-----------Columns types-----------")
object_cols,int_cols,float_cols=get_columns_types(train_df)
print("----------Missing value-----------")
missing_values_df=check_missing_data(train_df,"train_")
print("nbr of columns with missing values",len(missing_values_df))
print(check_missing_data(train_df,"train_"))
print("----------duplicated lines-----------")
print("are there any duplicated lines?")
print(train_df.drop(columns="Row_id").duplicated().any())
print("----------head-----------")
train_df.head()

----------Shape of Data----------
(356720, 37)
-----------Information-----------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356720 entries, 0 to 356719
Data columns (total 37 columns):
Row_id           356720 non-null int64
Hotel_id         356720 non-null int64
Date_id          356720 non-null int64
Date             356720 non-null object
LT               356720 non-null int64
BKG_0            356720 non-null float64
Revenue_0        356720 non-null float64
HotelPrice_0     356720 non-null float64
PriceComp1_0     356720 non-null float64
PriceComp2_0     356720 non-null float64
PriceComp3_0     356720 non-null float64
PriceComp4_0     356720 non-null float64
BKG_1            356720 non-null float64
Revenue_1        356720 non-null float64
BKG_7            356720 non-null float64
Revenue_7        356720 non-null float64
BKG_14           356720 non-null float64
Revenue_14       356720 non-null float64
HotelPrice_14    356720 non-null float64
PriceComp1_14    356720 non-null float

,Row_id,Hotel_id,Date_id,Date,LT,BKG_0,Revenue_0,HotelPrice_0,PriceComp1_0,PriceComp2_0,...,PriceComp2_21,PriceComp3_21,PriceComp4_21,BKG_28,Revenue_28,HotelPrice_28,PriceComp1_28,PriceComp2_28,PriceComp3_28,PriceComp4_28
0,1000001,1,43354,11/09/2018,37,2.796391,310.465881,209.117233,175.433029,254.684128,...,265.553040,185.216522,223.656219,2.712079,305.808167,241.821213,188.598557,259.636017,186.012878,222.797882
1,1000002,1,43319,07/08/2018,255,1.829377,267.324188,189.501221,179.592834,216.652176,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
2,1000003,1,43493,28/01/2019,298,0.000000,0.000000,165.191788,106.838440,155.952698,...,168.445602,158.099258,246.577759,0.000000,0.000000,173.946167,102.816086,205.642029,161.208801,260.023560
3,1000004,1,43371,28/09/2018,220,2.745681,498.146576,170.131302,0.000000,259.400360,...,267.351318,237.755630,250.307373,1.933594,348.409576,180.544403,0.000000,220.558350,229.740631,263.437622
4,1000005,1,43387,14/10/2018,136,0.925308,178.415405,184.628052,186.085831,244.755524,...,246.165451,181.919266,272.818390,0.913420,175.162933,179.204178,180.829193,248.956894,188.421494,260.718842


In [4]:
train_df=reduce_mem_usage(train_df)
test_df=reduce_mem_usage(test_df)
train_df.head()

Memory usage after optimization is: 28.24 MB
Decreased by 72.0%
Memory usage after optimization is: 11.81 MB
Decreased by 71.9%


,Row_id,Hotel_id,Date_id,Date,LT,BKG_0,Revenue_0,HotelPrice_0,PriceComp1_0,PriceComp2_0,...,PriceComp2_21,PriceComp3_21,PriceComp4_21,BKG_28,Revenue_28,HotelPrice_28,PriceComp1_28,PriceComp2_28,PriceComp3_28,PriceComp4_28
0,1000001,1,43354,11/09/2018,37,2.796875,310.500,209.125,175.3750,254.625,...,265.500,185.250,223.625,2.712891,305.750,241.875,188.6250,259.750,186.000,222.75
1,1000002,1,43319,07/08/2018,255,1.829102,267.250,189.500,179.6250,216.625,...,-1.000,-1.000,-1.000,-1.000000,-1.000,-1.000,-1.0000,-1.000,-1.000,-1.00
2,1000003,1,43493,28/01/2019,298,0.000000,0.000,165.250,106.8125,156.000,...,168.500,158.125,246.625,0.000000,0.000,174.000,102.8125,205.625,161.250,260.00
3,1000004,1,43371,28/09/2018,220,2.746094,498.250,170.125,0.0000,259.500,...,267.250,237.750,250.250,1.933594,348.500,180.500,0.0000,220.500,229.750,263.50
4,1000005,1,43387,14/10/2018,136,0.925293,178.375,184.625,186.1250,244.750,...,246.125,181.875,272.750,0.913574,175.125,179.250,180.8750,249.000,188.375,260.75


# generagte time features

In [5]:
train_df["Extraction_Date"]="1/1/1990"
test_df["Extraction_Date"]="1/1/1990"
train_df['Extraction_Date'] = train_df.apply(lambda x: decale_date(x['Date'], x['LT']), axis=1)
test_df['Extraction_Date'] = test_df.apply(lambda x: decale_date(x['Date'], x['LT']), axis=1)
train_df.head()

,Row_id,Hotel_id,Date_id,Date,LT,BKG_0,Revenue_0,HotelPrice_0,PriceComp1_0,PriceComp2_0,...,PriceComp3_21,PriceComp4_21,BKG_28,Revenue_28,HotelPrice_28,PriceComp1_28,PriceComp2_28,PriceComp3_28,PriceComp4_28,Extraction_Date
0,1000001,1,43354,11/09/2018,37,2.796875,310.500,209.125,175.3750,254.625,...,185.250,223.625,2.712891,305.750,241.875,188.6250,259.750,186.000,222.75,05/08/2018
1,1000002,1,43319,07/08/2018,255,1.829102,267.250,189.500,179.6250,216.625,...,-1.000,-1.000,-1.000000,-1.000,-1.000,-1.0000,-1.000,-1.000,-1.00,25/11/2017
2,1000003,1,43493,28/01/2019,298,0.000000,0.000,165.250,106.8125,156.000,...,158.125,246.625,0.000000,0.000,174.000,102.8125,205.625,161.250,260.00,05/04/2018
3,1000004,1,43371,28/09/2018,220,2.746094,498.250,170.125,0.0000,259.500,...,237.750,250.250,1.933594,348.500,180.500,0.0000,220.500,229.750,263.50,20/02/2018
4,1000005,1,43387,14/10/2018,136,0.925293,178.375,184.625,186.1250,244.750,...,181.875,272.750,0.913574,175.125,179.250,180.8750,249.000,188.375,260.75,31/05/2018


In [6]:
train_df=time_features1(train_df,"Date","rate")
test_df=time_features1(test_df,"Date","rate")
train_df=time_features2(train_df,"Date","extraction")
test_df=time_features2(test_df,"Date","extraction")
train_df.head()

,Row_id,Hotel_id,Date_id,Date,LT,BKG_0,Revenue_0,HotelPrice_0,PriceComp1_0,PriceComp2_0,...,quarter_rate,month_rate,dayofweek_extraction,dayofyear_extraction,dayofmonth_extraction,year_woy_extraction,is_weekend_extraction,year_extraction,quarter_extraction,month_extraction
0,1000001,1,43354,2018-09-11,37,2.796875,310.500,209.125,175.3750,254.625,...,3,9,1536624000000000000,254,11,201837,0,2018,3,9
1,1000002,1,43319,2018-08-07,255,1.829102,267.250,189.500,179.6250,216.625,...,3,8,1533600000000000000,219,7,201832,0,2018,3,8
2,1000003,1,43493,2019-01-28,298,0.000000,0.000,165.250,106.8125,156.000,...,1,1,1548633600000000000,28,28,20195,0,2019,1,1
3,1000004,1,43371,2018-09-28,220,2.746094,498.250,170.125,0.0000,259.500,...,3,9,1538092800000000000,271,28,201839,0,2018,3,9
4,1000005,1,43387,2018-10-14,136,0.925293,178.375,184.625,186.1250,244.750,...,4,10,1539475200000000000,287,14,201841,0,2018,4,10


# generate features from hotel revenue

In [7]:
bkg_cols=['BKG_0','BKG_1','BKG_7','BKG_14','BKG_21','BKG_28']
revenue_cols=['Revenue_0','Revenue_1','Revenue_7','Revenue_14','Revenue_21','Revenue_28']
price_lag_cols=["HotelPrice_14","HotelPrice_21","HotelPrice_28"]

In [8]:
train_df["r0-r1"]=train_df.Revenue_0-train_df.Revenue_1
test_df["r0-r1"]=test_df.Revenue_0-test_df.Revenue_1
train_df["r1-r7"]=train_df.Revenue_1-train_df.Revenue_7
test_df["r1-r7"]=test_df.Revenue_1-test_df.Revenue_7
# descriptive stats from revenue
train_df,test_df=generate_descriptive_stats(train_df,test_df,"Revenue",revenue_cols)
# revenue / booking TODO

train_df.head()

,Row_id,Hotel_id,Date_id,Date,LT,BKG_0,Revenue_0,HotelPrice_0,PriceComp1_0,PriceComp2_0,...,year_extraction,quarter_extraction,month_extraction,r0-r1,r1-r7,Revenue_mean,Revenue_max,Revenue_min,Revenue_median,Revenue_std
0,1000001,1,43354,2018-09-11,37,2.796875,310.500,209.125,175.3750,254.625,...,2018,3,9,19.500,-7.00,301.750,312.00,291.0,302.00,9.132812
1,1000002,1,43319,2018-08-07,255,1.829102,267.250,189.500,179.6250,216.625,...,2018,3,8,-18.250,286.50,91.500,285.50,-1.0,-1.00,143.375000
2,1000003,1,43493,2019-01-28,298,0.000000,0.000,165.250,106.8125,156.000,...,2019,1,1,0.000,0.00,0.000,0.00,0.0,0.00,0.000000
3,1000004,1,43371,2018-09-28,220,2.746094,498.250,170.125,0.0000,259.500,...,2018,3,9,24.750,-0.25,462.000,498.25,348.5,477.75,56.687500
4,1000005,1,43387,2018-10-14,136,0.925293,178.375,184.625,186.1250,244.750,...,2018,4,10,10.875,-11.00,174.625,178.50,167.5,175.75,4.234375


# generate features from Booking

In [9]:
train_df["bkg0-bkg1"]=train_df.BKG_0-train_df.BKG_1
test_df["bkg0-bkg1"]=test_df.BKG_0-test_df.BKG_1
train_df["bkg1-bkg7"]=train_df.BKG_1-train_df.BKG_7
test_df["bkg1-bkg7"]=test_df.BKG_1-test_df.BKG_7
# descriptive stats from revenue
train_df,test_df=generate_descriptive_stats(train_df,test_df,"BKG",bkg_cols)
train_df.head()

,Row_id,Hotel_id,Date_id,Date,LT,BKG_0,Revenue_0,HotelPrice_0,PriceComp1_0,PriceComp2_0,...,Revenue_min,Revenue_median,Revenue_std,bkg0-bkg1,bkg1-bkg7,BKG_mean,BKG_max,BKG_min,BKG_median,BKG_std
0,1000001,1,43354,2018-09-11,37,2.796875,310.500,209.125,175.3750,254.625,...,291.0,302.00,9.132812,-0.126953,0.029297,2.875000,2.966797,2.712891,2.910156,0.101257
1,1000002,1,43319,2018-08-07,255,1.829102,267.250,189.500,179.6250,216.625,...,-1.0,-1.00,143.375000,-0.135742,2.964844,-0.034515,1.964844,-1.000000,-1.000000,1.496094
2,1000003,1,43493,2019-01-28,298,0.000000,0.000,165.250,106.8125,156.000,...,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,1000004,1,43371,2018-09-28,220,2.746094,498.250,170.125,0.0000,259.500,...,348.5,477.75,56.687500,-0.183594,-0.027344,2.755859,2.988281,1.933594,2.943359,0.411865
4,1000005,1,43387,2018-10-14,136,0.925293,178.375,184.625,186.1250,244.750,...,167.5,175.75,4.234375,-0.033203,-0.021973,0.931152,0.980469,0.901855,0.919434,0.031250


# Booking - available rooms

In [10]:
hotels_capacity_df=pd.read_csv("hotels_capacities.csv",delimiter=";")
hotels_capacity_df.head()

,Hotel,Capacity
0,1,23
1,2,251
2,3,123
3,4,51


In [11]:
train_df = pd.merge(train_df, hotels_capacity_df,  how='left', left_on=['Hotel_id'], right_on = ['Hotel'])
test_df = pd.merge(test_df, hotels_capacity_df,  how='left', left_on=['Hotel_id'], right_on = ['Hotel'])
train_df.head()

,Row_id,Hotel_id,Date_id,Date,LT,BKG_0,Revenue_0,HotelPrice_0,PriceComp1_0,PriceComp2_0,...,Revenue_std,bkg0-bkg1,bkg1-bkg7,BKG_mean,BKG_max,BKG_min,BKG_median,BKG_std,Hotel,Capacity
0,1000001,1,43354,2018-09-11,37,2.796875,310.500,209.125,175.3750,254.625,...,9.132812,-0.126953,0.029297,2.875000,2.966797,2.712891,2.910156,0.101257,1,23
1,1000002,1,43319,2018-08-07,255,1.829102,267.250,189.500,179.6250,216.625,...,143.375000,-0.135742,2.964844,-0.034515,1.964844,-1.000000,-1.000000,1.496094,1,23
2,1000003,1,43493,2019-01-28,298,0.000000,0.000,165.250,106.8125,156.000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,23
3,1000004,1,43371,2018-09-28,220,2.746094,498.250,170.125,0.0000,259.500,...,56.687500,-0.183594,-0.027344,2.755859,2.988281,1.933594,2.943359,0.411865,1,23
4,1000005,1,43387,2018-10-14,136,0.925293,178.375,184.625,186.1250,244.750,...,4.234375,-0.033203,-0.021973,0.931152,0.980469,0.901855,0.919434,0.031250,1,23


In [12]:
train_df["capacity-BKG"]=train_df.Capacity - train_df.BKG_0
train_df["capacity-sum_BKG"]=train_df.Capacity - train_df[bkg_cols].sum(axis=1)
test_df["capacity-BKG"]=test_df.Capacity - test_df.BKG_0
test_df["capacity-sum_BKG"]=test_df.Capacity - test_df[bkg_cols].sum(axis=1)
train_df.head()

,Row_id,Hotel_id,Date_id,Date,LT,BKG_0,Revenue_0,HotelPrice_0,PriceComp1_0,PriceComp2_0,...,bkg1-bkg7,BKG_mean,BKG_max,BKG_min,BKG_median,BKG_std,Hotel,Capacity,capacity-BKG,capacity-sum_BKG
0,1000001,1,43354,2018-09-11,37,2.796875,310.500,209.125,175.3750,254.625,...,0.029297,2.875000,2.966797,2.712891,2.910156,0.101257,1,23,20.203125,5.750000
1,1000002,1,43319,2018-08-07,255,1.829102,267.250,189.500,179.6250,216.625,...,2.964844,-0.034515,1.964844,-1.000000,-1.000000,1.496094,1,23,21.170898,23.206055
2,1000003,1,43493,2019-01-28,298,0.000000,0.000,165.250,106.8125,156.000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,23,23.000000,23.000000
3,1000004,1,43371,2018-09-28,220,2.746094,498.250,170.125,0.0000,259.500,...,-0.027344,2.755859,2.988281,1.933594,2.943359,0.411865,1,23,20.253906,6.468750
4,1000005,1,43387,2018-10-14,136,0.925293,178.375,184.625,186.1250,244.750,...,-0.021973,0.931152,0.980469,0.901855,0.919434,0.031250,1,23,22.074707,17.410156


# generate features from price lag:

In [13]:
train_df["price_lag_14-price_lag_21"]=train_df.HotelPrice_14-train_df.HotelPrice_21
test_df["price_lag_14-price_lag_21"]=test_df.HotelPrice_14-test_df.HotelPrice_21
train_df["price_lag_21-price_lag_28"]=train_df.HotelPrice_21-train_df.HotelPrice_28
test_df["price_lag_21-price_lag_28"]=test_df.HotelPrice_21-test_df.HotelPrice_28
# descriptive stats from revenue
train_df,test_df=generate_descriptive_stats(train_df,test_df,"HotelPrice",price_lag_cols)
# calculate avg lag prices by giving more weight to the closest lag to reservation data:
train_df["price_lag_mean_with_weights"]=train_df.HotelPrice_14*0.6 + train_df.HotelPrice_21*0.3 + train_df.HotelPrice_28*0.1
test_df["price_lag_mean_with_weights"]=test_df.HotelPrice_14*0.6 + test_df.HotelPrice_21*0.3 + test_df.HotelPrice_28*0.1

train_df.head()

,Row_id,Hotel_id,Date_id,Date,LT,BKG_0,Revenue_0,HotelPrice_0,PriceComp1_0,PriceComp2_0,...,capacity-BKG,capacity-sum_BKG,price_lag_14-price_lag_21,price_lag_21-price_lag_28,HotelPrice_mean,HotelPrice_max,HotelPrice_min,HotelPrice_median,HotelPrice_std,price_lag_mean_with_weights
0,1000001,1,43354,2018-09-11,37,2.796875,310.500,209.125,175.3750,254.625,...,20.203125,5.750000,-19.00,-17.750,223.625,241.875,205.125,224.125,18.375000,214.500
1,1000002,1,43319,2018-08-07,255,1.829102,267.250,189.500,179.6250,216.625,...,21.170898,23.206055,0.00,0.000,-1.000,-1.000,-1.000,-1.000,0.000000,-1.000
2,1000003,1,43493,2019-01-28,298,0.000000,0.000,165.250,106.8125,156.000,...,23.000000,23.000000,-5.75,-0.250,172.000,174.000,168.000,173.750,3.394531,170.375
3,1000004,1,43371,2018-09-28,220,2.746094,498.250,170.125,0.0000,259.500,...,20.253906,6.468750,4.75,-13.125,173.375,180.500,167.375,172.125,6.644531,171.500
4,1000005,1,43387,2018-10-14,136,0.925293,178.375,184.625,186.1250,244.750,...,22.074707,17.410156,4.75,-1.125,180.000,182.875,178.125,179.250,2.482422,181.125


# generate features from competitors:

In [14]:
competitor1_cols=["PriceComp1_0","PriceComp1_14","PriceComp1_21","PriceComp1_28"]
competitor2_cols=["PriceComp2_0","PriceComp2_14","PriceComp2_21","PriceComp2_28"]
competitor3_cols=["PriceComp3_0","PriceComp3_14","PriceComp3_21","PriceComp3_28"]
competitor4_cols=["PriceComp4_0","PriceComp4_14","PriceComp4_21","PriceComp4_28"]
all_competitors_cols=competitor1_cols +  competitor2_cols + competitor3_cols + competitor4_cols

In [15]:
# descriptive statistics
train_df,test_df=generate_descriptive_stats(train_df,test_df,"competitor1",competitor1_cols)
train_df,test_df=generate_descriptive_stats(train_df,test_df,"competitor2",competitor2_cols)
train_df,test_df=generate_descriptive_stats(train_df,test_df,"competitor3",competitor3_cols)
train_df,test_df=generate_descriptive_stats(train_df,test_df,"competitor4",competitor4_cols)
train_df,test_df=generate_descriptive_stats(train_df,test_df,"all_competitors",all_competitors_cols)
train_df.head()

,Row_id,Hotel_id,Date_id,Date,LT,BKG_0,Revenue_0,HotelPrice_0,PriceComp1_0,PriceComp2_0,...,competitor4_mean,competitor4_max,competitor4_min,competitor4_median,competitor4_std,all_competitors_mean,all_competitors_max,all_competitors_min,all_competitors_median,all_competitors_std
0,1000001,1,43354,2018-09-11,37,2.796875,310.500,209.125,175.3750,254.625,...,218.50000,227.875,200.125,223.25,12.515625,210.62500,265.500,170.000,197.750,32.78125
1,1000002,1,43319,2018-08-07,255,1.829102,267.250,189.500,179.6250,216.625,...,62.09375,251.375,-1.000,-1.00,126.187500,49.71875,251.375,-1.000,-1.000,92.50000
2,1000003,1,43493,2019-01-28,298,0.000000,0.000,165.250,106.8125,156.000,...,252.50000,260.000,246.625,251.75,6.843750,172.50000,260.000,100.875,160.125,55.71875
3,1000004,1,43371,2018-09-28,220,2.746094,498.250,170.125,0.0000,259.500,...,255.62500,263.500,250.250,254.25,5.796875,185.75000,267.250,0.000,236.750,111.56250
4,1000005,1,43387,2018-10-14,136,0.925293,178.375,184.625,186.1250,244.750,...,270.75000,283.000,260.750,269.50,9.601562,221.50000,283.000,179.250,216.750,38.68750


In [16]:
# difference of price with competitors
for i in range(1,5):
    train_df["price_diff_with_comp_"+str(i)+"_14"] = train_df["HotelPrice_14"]-train_df["PriceComp"+str(i)+"_14"]
    test_df["price_diff_with_comp_"+str(i)+"_14"] = test_df["HotelPrice_14"]-test_df["PriceComp"+str(i)+"_14"]
    train_df["price_diff_with_comp_"+str(i)+"_21"] = train_df["HotelPrice_21"]-train_df["PriceComp"+str(i)+"_21"]
    test_df["price_diff_with_comp_"+str(i)+"_21"] = test_df["HotelPrice_21"]-test_df["PriceComp"+str(i)+"_21"]
    train_df["price_diff_with_comp_"+str(i)+"_28"] = train_df["HotelPrice_28"]-train_df["PriceComp"+str(i)+"_28"]
    test_df["price_diff_with_comp_"+str(i)+"_28"] = test_df["HotelPrice_28"]-test_df["PriceComp"+str(i)+"_28"]
train_df.head()

,Row_id,Hotel_id,Date_id,Date,LT,BKG_0,Revenue_0,HotelPrice_0,PriceComp1_0,PriceComp2_0,...,price_diff_with_comp_1_28,price_diff_with_comp_2_14,price_diff_with_comp_2_21,price_diff_with_comp_2_28,price_diff_with_comp_3_14,price_diff_with_comp_3_21,price_diff_with_comp_3_28,price_diff_with_comp_4_14,price_diff_with_comp_4_21,price_diff_with_comp_4_28
0,1000001,1,43354,2018-09-11,37,2.796875,310.500,209.125,175.3750,254.625,...,53.2500,-45.625,-41.375,-17.875,22.250,38.875,55.875,-22.750,0.500,19.125
1,1000002,1,43319,2018-08-07,255,1.829102,267.250,189.500,179.6250,216.625,...,0.0000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,1000003,1,43493,2019-01-28,298,0.000000,0.000,165.250,106.8125,156.000,...,71.1875,-0.500,5.250,-31.625,9.000,15.625,12.750,-88.750,-72.875,-86.000
3,1000004,1,43371,2018-09-28,220,2.746094,498.250,170.125,0.0000,259.500,...,180.5000,-92.375,-99.875,-40.000,-62.625,-70.375,-49.250,-83.875,-82.875,-83.000
4,1000005,1,43387,2018-10-14,136,0.925293,178.375,184.625,186.1250,244.750,...,-1.6250,-54.125,-68.000,-69.750,-0.125,-3.750,-9.125,-100.125,-94.625,-81.500


In [17]:
def train_model(X_train,Y_train,X_val,Y_val,X_test,parmaters,features_name): 
    d_train = xgb.DMatrix(X_train, Y_train,feature_names=features_name)
    d_valid = xgb.DMatrix(X_val, Y_val,feature_names=features_name)
    d_test = xgb.DMatrix(X_test,feature_names=features_name)
    list_track = [(d_train, 'train'), (d_valid, 'valid')]
    model = xgb.train(parmaters, d_train, 3500,  list_track, maximize=False, verbose_eval=50, early_stopping_rounds=50)
    train_pred =model.predict(d_train)              
    valid_pred =model.predict(d_valid)   
    test_pred = model.predict(d_test)
    return train_pred ,valid_pred,test_pred,model

def train_kfold(X_train,Y_train,X_test,parmaters,features_name,split=5):
    final_train_pred=np.zeros_like(Y_train)
    final_test_pred=np.zeros(len(X_test))
    
    kf = StratifiedKFold(n_splits=split,random_state=2222)
    i=1
    for train_index, val_index in kf.split(X_train, X_train["Hotel_id"]):
        print("fold:"+str(i))
        train_fold_features, val_fold_features = X_train.loc[train_index], X_train.loc[val_index]
        train_fold_target, val_fold_target = Y_train.loc[train_index], Y_train.loc[val_index] 
        train_pred ,valid_pred,test_pred,model=train_model( 
                                                        X_train=train_fold_features,
                                                        Y_train= train_fold_target,
                                                        X_val= val_fold_features,
                                                        Y_val= val_fold_target,
                                                        X_test= X_test,
                                                        parmaters=parmaters,
                                                        features_name=features_name 
                                                    )
        
        final_train_pred[val_index]=valid_pred
        final_test_pred=final_test_pred+(test_pred/split)
        i=i+1
    return final_train_pred,final_test_pred,model

def train_Xgboost(train_df,test_df,Y,target,metric="rmse") :
    
    import gc # garbej collector for mempry optimisation
    gc.enable()
    from sklearn.metrics import accuracy_score # to be changed in case of AUC,...
    from sklearn.metrics import roc_auc_score,mean_squared_error
    from sklearn.model_selection import train_test_split    
    from sklearn.metrics import mean_absolute_error

    X=train_df.drop(columns=[target])
    #Y=train_df[target]
    X_train,X_test,y_train,y_test=train_test_split(X, Y, test_size=0.2, random_state=42,stratify=X["Hotel_id"])
    #use this in case classification
    '''X_train,X_test,y_train,y_test=train_test_split(X, Y, test_size=0.33, stratify=Y,random_state=42)'''

    
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    
    # "Learn" the mean from the training data
    mean_train = np.mean(y_train)
    # Get predictions on the test set
    baseline_predictions = np.ones(y_test.shape) * mean_train
    # Compute MAE
    mae_baseline = mean_absolute_error(y_test, baseline_predictions)
    
    print("Baseline {} is {:.2f}".format(metric,mae_baseline))
    
    params = {
        # Parameters that we are going to tune.
        'max_depth':7,
        'min_child_weight': 1,
        'eta':.3,
        'subsample': 1,
        'colsample_bytree': 1,
        # Other parameters
        'objective':'reg:linear'
    }
    params['eval_metric'] = metric
    num_boost_round = 3000
    
    
    model = xgb.train(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        evals=[(dtest, "Test")],
        early_stopping_rounds=50
    )
    print("Best {} with default parameters : {:.2f} with {} rounds".format(
                metric, 
                model.best_score,
                 model.best_iteration+1))
    
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={metric},
        early_stopping_rounds=50
    )
    print("Best {} with cv : {}".format(metric,cv_results['test-'+metric+'-mean'].min()))
    
    
    print('--Tunning Parameters max_depth and min_child_weight--')
    # Define initial best params and MAE
    min_mae = float("Inf")
    best_params = None
    
    gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(7,14)# for large dataset and features this can go to 20
    for min_child_weight in [2,4,5,6,7]
    ]

    for max_depth, min_child_weight in gridsearch_params:
        print("CV with max_depth={}, min_child_weight={}".format(
                                 max_depth,
                                 min_child_weight))
        # Update our parameters
        params['max_depth'] = max_depth
        params['min_child_weight'] = min_child_weight
        # Run CV
        cv_results = xgb.cv(
            params,
            dtrain,
            num_boost_round=num_boost_round,
            seed=42,
            nfold=5,
            metrics={metric},
            early_stopping_rounds=50
        )
        # Update best MAE
        mean_mae = cv_results['test-'+metric+'-mean'].min()
        boost_rounds = cv_results['test-'+metric+'-mean'].argmin()
        print("\t {} {} for {} rounds".format(metric,mean_mae, boost_rounds))
        if mean_mae < min_mae:
            min_mae = mean_mae
            best_params = (max_depth,min_child_weight)
        
    params['max_depth'] = best_params[0]
    params['min_child_weight'] = best_params[1]
    print("best params: max_depth={} and min_child_weight={}".format(params['max_depth'],params['min_child_weight']))
    print('--Tunning Parameters subsample and colsample_bytree--')
    gridsearch_params = [
        (subsample, colsample)
        for subsample in [i/10. for i in range(6,11)]
        for colsample in [i/10. for i in range(3,11)]
    ]
    min_mae = float("Inf")
    best_params = None
    # We start by the largest values and go down to the smallest
    for subsample, colsample in reversed(gridsearch_params):
        print("CV with subsample={}, colsample={}".format(
                                 subsample,
                                 colsample))
        # We update our parameters
        params['subsample'] = subsample
        params['colsample_bytree'] = colsample
        # Run CV
        cv_results = xgb.cv(
            params,
            dtrain,
            num_boost_round=num_boost_round,
            seed=42,
            nfold=5,
            metrics={metric},
            early_stopping_rounds=50
        )
        # Update best score
        mean_mae = cv_results['test-'+metric+'-mean'].min()
        boost_rounds = cv_results['test-'+metric+'-mean'].argmin()
        print("\t {} {} for {} rounds".format(metric,mean_mae, boost_rounds))
        if mean_mae < min_mae:
            min_mae = mean_mae
            best_params = (subsample,colsample)
    print("Best params: subsample={}, colsample_bytree={}, {}: {}".format(best_params[0], best_params[1],metric, min_mae))
    params['subsample'] = best_params[0]
    params['colsample_bytree'] = best_params[1]
    best_params=0.1
    for eta in [.2, .1,.07, .05, .01, .005]:
        print("CV with eta={}".format(eta))
        # We update our parameters
        params['eta'] = eta
        # Run and time CV
        cv_results = xgb.cv(
            params,
            dtrain,
            num_boost_round=num_boost_round,
            seed=42,
            nfold=5,
            metrics=[metric],
            early_stopping_rounds=50
              )
        # Update best score
        mean_mae = cv_results['test-'+metric+'-mean'].min()
        boost_rounds = cv_results['test-'+metric+'-mean'].argmin()
        print("\t{} {} for {} rounds\n".format(metric,mean_mae, boost_rounds))
        if mean_mae < min_mae:
            min_mae = mean_mae
            best_params = eta
    print("Best params: eta={}, {}: {}".format(best_params, metric, min_mae))
    params['eta'] = best_params
    print("Final Best params: {}".format(params))
    
    # train and predict using your best model
    features_name=X.columns
    train_pred,test_pred,model= train_kfold(X,Y,test_df,params,features_name,split=5)
    return train_pred,test_pred,params,model

In [18]:
X=train_df.drop(columns=columns_to_drop)
X=X.drop(columns=["HotelPrice_0"])
y=train_df.HotelPrice_0
X_test=test_df.drop(columns=columns_to_drop)
X.head()

,Hotel_id,Date_id,LT,BKG_0,Revenue_0,PriceComp1_0,PriceComp2_0,PriceComp3_0,PriceComp4_0,BKG_1,...,price_diff_with_comp_1_28,price_diff_with_comp_2_14,price_diff_with_comp_2_21,price_diff_with_comp_2_28,price_diff_with_comp_3_14,price_diff_with_comp_3_21,price_diff_with_comp_3_28,price_diff_with_comp_4_14,price_diff_with_comp_4_21,price_diff_with_comp_4_28
0,1,43354,37,2.796875,310.500,175.3750,254.625,170.000,200.125,2.923828,...,53.2500,-45.625,-41.375,-17.875,22.250,38.875,55.875,-22.750,0.500,19.125
1,1,43319,255,1.829102,267.250,179.6250,216.625,159.500,251.375,1.964844,...,0.0000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,1,43493,298,0.000000,0.000,106.8125,156.000,155.875,246.875,0.000000,...,71.1875,-0.500,5.250,-31.625,9.000,15.625,12.750,-88.750,-72.875,-86.000
3,1,43371,220,2.746094,498.250,0.0000,259.500,235.625,252.500,2.929688,...,180.5000,-92.375,-99.875,-40.000,-62.625,-70.375,-49.250,-83.875,-82.875,-83.000
4,1,43387,136,0.925293,178.375,186.1250,244.750,191.375,266.000,0.958496,...,-1.6250,-54.125,-68.000,-69.750,-0.125,-3.750,-9.125,-100.125,-94.625,-81.500


In [20]:
final_test_pred

array([184.60589218, 150.62909698, 216.95775604, ...,  80.29709435,
        84.17543983,  84.10102463])

In [19]:
features_name=X.columns
tmp_params = {
       # Parameters that we are going to tune.
       'booster': 'gbtree',
       'max_depth':7,
       'min_child_weight': 1,
       'eta':.2,
       'subsample': 1,
       'colsample_bytree': 1,
       # Other parameters
       'objective':'reg:linear'
   }
tmp_params['eval_metric'] = 'rmse'
final_train_pred,final_test_pred,model=train_kfold(X,y,X_test,tmp_params,features_name,split=3)

fold:1
[15:46:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 172 extra nodes, 0 pruned nodes, max_depth=7
[0]	train-rmse:100.501	valid-rmse:100.511
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 50 rounds.
[15:46:54] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 156 extra nodes, 0 pruned nodes, max_depth=7
[15:46:54] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 182 extra nodes, 0 pruned nodes, max_depth=7
[15:46:55] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 186 extra nodes, 0 pruned nodes, max_depth=7
[15:46:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 202 extra nodes, 0 pruned nodes, max_depth=7
[15:46:57] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 200 extra nodes, 0 pruned nodes, max_depth=7
[15:46:57] /workspace/src/tree/updater_prune.cc:74: tree pruni

[15:47:40] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 202 extra nodes, 0 pruned nodes, max_depth=7
[15:47:41] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 246 extra nodes, 0 pruned nodes, max_depth=7
[15:47:42] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 204 extra nodes, 0 pruned nodes, max_depth=7
[15:47:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 208 extra nodes, 0 pruned nodes, max_depth=7
[15:47:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 210 extra nodes, 0 pruned nodes, max_depth=7
[15:47:44] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 196 extra nodes, 0 pruned nodes, max_depth=7
[15:47:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 160 extra nodes, 0 pruned nodes, max_depth=7
[15:47:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 216 extra nodes, 0 pruned nodes, max_depth=7
[15:47:4

[15:48:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 226 extra nodes, 0 pruned nodes, max_depth=7
[15:48:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 154 extra nodes, 0 pruned nodes, max_depth=7
[15:48:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 168 extra nodes, 0 pruned nodes, max_depth=7
[15:48:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 124 extra nodes, 0 pruned nodes, max_depth=7
[15:48:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 164 extra nodes, 0 pruned nodes, max_depth=7
[15:48:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 156 extra nodes, 0 pruned nodes, max_depth=7
[15:48:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=7
[15:48:34] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 176 extra nodes, 0 pruned nodes, max_depth=7
[15:48:3

[15:49:16] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 194 extra nodes, 0 pruned nodes, max_depth=7
[15:49:16] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 186 extra nodes, 0 pruned nodes, max_depth=7
[15:49:17] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 98 extra nodes, 0 pruned nodes, max_depth=7
[15:49:18] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 238 extra nodes, 0 pruned nodes, max_depth=7
[200]	train-rmse:9.45989	valid-rmse:13.0336
[15:49:18] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 210 extra nodes, 0 pruned nodes, max_depth=7
[15:49:19] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 208 extra nodes, 0 pruned nodes, max_depth=7
[15:49:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 172 extra nodes, 0 pruned nodes, max_depth=7
[15:49:21] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 142 extra 

[15:50:02] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 234 extra nodes, 0 pruned nodes, max_depth=7
[15:50:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 146 extra nodes, 0 pruned nodes, max_depth=7
[15:50:04] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 174 extra nodes, 0 pruned nodes, max_depth=7
[15:50:04] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 222 extra nodes, 0 pruned nodes, max_depth=7
[15:50:05] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 166 extra nodes, 0 pruned nodes, max_depth=7
[15:50:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 182 extra nodes, 0 pruned nodes, max_depth=7
[15:50:07] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 208 extra nodes, 0 pruned nodes, max_depth=7
[15:50:07] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 138 extra nodes, 0 pruned nodes, max_depth=7
[15:50:0

...

[17:25:14] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 130 extra nodes, 0 pruned nodes, max_depth=7
[17:25:15] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 194 extra nodes, 0 pruned nodes, max_depth=7
[17:25:15] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 196 extra nodes, 0 pruned nodes, max_depth=7
[17:25:16] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 218 extra nodes, 0 pruned nodes, max_depth=7
[17:25:17] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 230 extra nodes, 0 pruned nodes, max_depth=7
[17:25:17] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 176 extra nodes, 0 pruned nodes, max_depth=7
[17:25:18] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 212 extra nodes, 0 pruned nodes, max_depth=7
[17:25:19] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 150 extra nodes, 0 pruned nodes, max_depth=7
[17:25:2

[17:26:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 142 extra nodes, 0 pruned nodes, max_depth=7
[17:26:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 136 extra nodes, 0 pruned nodes, max_depth=7
[17:26:01] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 216 extra nodes, 0 pruned nodes, max_depth=7
[17:26:02] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 192 extra nodes, 0 pruned nodes, max_depth=7
[17:26:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 168 extra nodes, 0 pruned nodes, max_depth=7
[17:26:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 198 extra nodes, 0 pruned nodes, max_depth=7
[17:26:04] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 168 extra nodes, 0 pruned nodes, max_depth=7
[17:26:05] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 172 extra nodes, 0 pruned nodes, max_depth=7
[17:26:0

[17:26:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 208 extra nodes, 0 pruned nodes, max_depth=7
[1950]	train-rmse:2.98045	valid-rmse:11.6027
[17:26:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 210 extra nodes, 0 pruned nodes, max_depth=7
[17:26:47] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 194 extra nodes, 0 pruned nodes, max_depth=7
[17:26:47] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 172 extra nodes, 0 pruned nodes, max_depth=7
[17:26:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 152 extra nodes, 0 pruned nodes, max_depth=7
[17:26:49] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 160 extra nodes, 0 pruned nodes, max_depth=7
[17:26:49] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 184 extra nodes, 0 pruned nodes, max_depth=7
[17:26:50] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra

[17:27:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 230 extra nodes, 0 pruned nodes, max_depth=7
[17:27:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 174 extra nodes, 0 pruned nodes, max_depth=7
[17:27:34] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 144 extra nodes, 0 pruned nodes, max_depth=7
[17:27:34] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 162 extra nodes, 0 pruned nodes, max_depth=7
[17:27:35] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 202 extra nodes, 0 pruned nodes, max_depth=7
[17:27:36] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 152 extra nodes, 0 pruned nodes, max_depth=7
[17:27:36] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 172 extra nodes, 0 pruned nodes, max_depth=7
[17:27:37] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 164 extra nodes, 0 pruned nodes, max_depth=7
[17:27:3

[17:28:18] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 152 extra nodes, 0 pruned nodes, max_depth=7
[17:28:18] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 198 extra nodes, 0 pruned nodes, max_depth=7
[17:28:19] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 148 extra nodes, 0 pruned nodes, max_depth=7
[17:28:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 140 extra nodes, 0 pruned nodes, max_depth=7
[17:28:21] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 164 extra nodes, 0 pruned nodes, max_depth=7
[17:28:21] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 214 extra nodes, 0 pruned nodes, max_depth=7
[17:28:22] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 218 extra nodes, 0 pruned nodes, max_depth=7
[17:28:23] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 154 extra nodes, 0 pruned nodes, max_depth=7
[17:28:2